Data source: 
- [Edger allan](https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt)
- [Robert Frost](https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [3]:
input_files = [
  r'D:\Notes and Exercises\Machine-Learning\dataset\edgar_allan_poe.txt',
  r'D:\Notes and Exercises\Machine-Learning\dataset\robert_frost.txt',
]

In [8]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

D:\Notes and Exercises\Machine-Learning\dataset\edgar_allan_poe.txt corresponds to label 0
D:\Notes and Exercises\Machine-Learning\dataset\robert_frost.txt corresponds to label 1


In [9]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [10]:
len(Ytrain), len(Ytest)

(1618, 540)

In [11]:
train_text[:5]

['their arthur amy was the father of mine',
 'to spare them any trouble double troubles',
 'inscription for a garden wall',
 'and sometimes something seemed to yield',
 'a shelfs for a clock or vase or picture']

In [12]:
Ytrain[:5]

[1, 1, 1, 1, 1]

In [13]:
idx = 1
word2idx = {'<unk>': 0}

In [14]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [15]:
word2idx

{'<unk>': 0,
 'their': 1,
 'arthur': 2,
 'amy': 3,
 'was': 4,
 'the': 5,
 'father': 6,
 'of': 7,
 'mine': 8,
 'to': 9,
 'spare': 10,
 'them': 11,
 'any': 12,
 'trouble': 13,
 'double': 14,
 'troubles': 15,
 'inscription': 16,
 'for': 17,
 'a': 18,
 'garden': 19,
 'wall': 20,
 'and': 21,
 'sometimes': 22,
 'something': 23,
 'seemed': 24,
 'yield': 25,
 'shelfs': 26,
 'clock': 27,
 'or': 28,
 'vase': 29,
 'picture': 30,
 'tell': 31,
 'me': 32,
 'where': 33,
 'youre': 34,
 'off': 35,
 'forâ€”montreal': 36,
 'in': 37,
 'seraphic': 38,
 'glancing': 39,
 'thine': 40,
 'eyes': 41,
 'smarty': 42,
 'someone': 43,
 'who': 44,
 'spoiled': 45,
 'everything': 46,
 'be': 47,
 'ready': 48,
 'i': 49,
 'thought': 50,
 'almost': 51,
 'anything': 52,
 'assailed': 53,
 'monarchs': 54,
 'high': 55,
 'estate': 56,
 'its': 57,
 'raining': 58,
 'by': 59,
 'thee': 60,
 'poetry': 61,
 'thy': 62,
 'presence': 63,
 'passionate': 64,
 'light': 65,
 'such': 66,
 'his': 67,
 'spirit': 68,
 'fit': 69,
 'shes': 70,
 '

In [16]:
len(word2idx)

2510

In [17]:

# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [18]:
train_text_int[100:105]

[[7, 5, 397],
 [230, 18, 398, 399, 7, 400, 34, 401, 121],
 [402, 403, 404, 175, 405],
 [406, 37, 407, 165, 408, 347, 409],
 [21, 410, 112, 68, 411, 105, 37, 5, 412]]

In [19]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [20]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [21]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [22]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [23]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3374536464771323, 0.6625463535228677)

In [24]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]
      
      # update last_idx
      last_idx = idx
    
    return logprob
  
  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [25]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [26]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9975278121137207


In [27]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8037037037037037


In [28]:
from sklearn.metrics import confusion_matrix, f1_score

# read about F-score: https://en.wikipedia.org/wiki/F-score

In [29]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 542,    4],
       [   0, 1072]], dtype=int64)

In [30]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 85,  91],
       [ 15, 349]], dtype=int64)

In [31]:
f1_score(Ytrain, Ptrain)

0.9981378026070763

In [32]:
f1_score(Ytest, Ptest)

0.8681592039800996